In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
df=pd.read_csv('price_22.csv')

In [3]:
df.head()

,Torque,Power,ARAI_Certified_Mileage,Ex-Showroom_Price
0,51Nm@4000rpm,38PS@5500rpm,23.6 km/litre,"Rs. 2,92,667"
1,51Nm@4000rpm,38PS@5500rpm,23.6 km/litre,"Rs. 2,36,447"
2,51Nm@4000rpm,38PS@5500rpm,NaN,"Rs. 2,96,661"
3,51Nm@4000rpm,38PS@5500rpm,21.9 km/litre,"Rs. 3,34,768"
4,51Nm@4000rpm,38PS@5500rpm,23.6 km/litre,"Rs. 2,72,223"


In [4]:
df['Torque']=df['Torque'].fillna('90Nm@3500rpm')
df['Ex-Showroom_Price'] = df['Ex-Showroom_Price'].str.replace('Rs. ','',regex=False)
df['Ex-Showroom_Price'] = df['Ex-Showroom_Price'].str.replace(',','',regex=False)
df['Ex-Showroom_Price'] = df['Ex-Showroom_Price'].astype(int)
df.loc[df.ARAI_Certified_Mileage == '9.8-10.0 km/litre','ARAI_Certified_Mileage'] = '10'
df.loc[df.ARAI_Certified_Mileage == '22.4-21.9 km/litre','ARAI_Certified_Mileage'] = '20'
df.loc[df.ARAI_Certified_Mileage == '10kmpl km/litre','ARAI_Certified_Mileage'] = '10'
df['ARAI_Certified_Mileage'] = df['ARAI_Certified_Mileage'].str.replace(' km/litre','',regex=False).astype(float)
df['ARAI_Certified_Mileage']=df['ARAI_Certified_Mileage'].fillna(23.00)
HP = df.Power.str.extract(r'(\d{1,4}).*').astype(int) * 0.98632
HP = HP.apply(lambda x: round(x,2))
TQ = df.Torque.str.extract(r'(\d{1,4}).*').astype(int)
TQ = TQ.apply(lambda x: round(x,2))
df.Torque = TQ
df.Power = HP

In [5]:
df.head()

,Torque,Power,ARAI_Certified_Mileage,Ex-Showroom_Price
0,51,37.48,23.6,292667
1,51,37.48,23.6,236447
2,51,37.48,23.0,296661
3,51,37.48,21.9,334768
4,51,37.48,23.6,272223


In [6]:
df.isnull().sum()

Torque                    0
Power                     0
ARAI_Certified_Mileage    0
Ex-Showroom_Price         0
dtype: int64

In [7]:
from sklearn.linear_model import LogisticRegression  # for Logistic Regression algorithm
from sklearn.model_selection import train_test_split #to split the dataset for training and testing
from sklearn.neighbors import KNeighborsClassifier  # for K nearest neighbours
from sklearn import svm  #for Support Vector Machine (SVM) Algorithm
from sklearn import metrics #for checking the model accuracy
from sklearn.tree import DecisionTreeClassifier #for using Decision Tree Algoithm

In [8]:
train, test = train_test_split(df, test_size = 0.3)
print(train.shape)
print(test.shape)

(893, 4)
(383, 4)


In [9]:
train_X = train[['Torque', 'Power', 'ARAI_Certified_Mileage']]
train_y=train['Ex-Showroom_Price']
test_X= test[['Torque', 'Power', 'ARAI_Certified_Mileage']]
test_y =test['Ex-Showroom_Price']

In [10]:
model = svm.SVC() 
model.fit(train_X,train_y) 
prediction=model.predict(test_X)

In [11]:
print('The accuracy of the SVM is:',metrics.accuracy_score(prediction,test_y))

The accuracy of the SVM is: 0.0026109660574412533


In [12]:
print(model.predict([[0,1,4]]))

[999900]


In [13]:
pickle.dump(model, open('Price_22_Pred_Model.pkl','wb'))